In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

In [5]:
# Ticket prices dictionary
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "tunis": "$120", "barcelone": "$70"}

In [6]:
# Flight status dictionary
flight_statuses = {
    "london": "On Time",
    "paris": "Delayed by 30 minutes", 
    "tokyo": "Boarding", 
    "berlin": "Arrived", 
    "tunis": "Cancelled", 
    "barcelone": "On Time"
}

In [7]:
# Available flight routes
available_routes = {
    "europe": ["london", "paris", "berlin", "barcelone"],
    "asia": ["tokyo"],
    "africa": ["tunis"],
    "direct_routes": {
        "london": ["paris", "berlin", "barcelone"],
        "paris": ["london", "berlin"],
        "tokyo": [],  # No direct routes from Tokyo in this example
        "berlin": ["london", "paris"],
        "tunis": [],
        "barcelone": ["london"]
    }
}

In [8]:
# Ticket Price Tool Function
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [9]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [10]:
# Flight Status Tool Function
def get_flight_status(destination_city):
    print(f"tool get_flight_status called for {destination_city}")
    city =  destination_city.lower()
    return flight_statuses.get(city, "Status not available")

In [11]:
# Available Routes Tool Function
def get_available_routes(region_or_city=None):
    print("fTool get_available_routes called for {region_or_city}")
    
    #If no specific region/city is provided, return all regions 
    if not region_or_city:
        return json.dumps({
            "regions": list(available_routes.keys() - {"direct_routes"}),
            "all_destinations": list(ticket_prices.keys())
        })
    
    region_or_city = region_or_city.lower()
    
    #check if it's a region 
    if region_or_city in available_routes and region_or_city != "direct_routes":
        return json.dumps(available_routes[region_or_city])
    
    #check of it's a city with direct routes 
    if region_or_city in available_routes["direct_routes"]:
        return json.dumps({
            "city": region_or_city,
            "direct_routes": available_routes["direct_routes"][region_or_city]
        })
    
    return "No routes found for the specified region or city"
        

In [12]:
# Ticket Price Function Definition for OpenAI
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [13]:
# Flight Status Function Definition for OpenAI
status_function = {
    "name": "get_flight_status",
    "description": "Get the current status of a flight to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city":{
                "type": "string",
                "description": "The city of the flight whose status is being queried.", 
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
    
}

In [14]:
# Available Routes Function Definition for OpenAI
routes_function = {
    "name": "get_available_routes",
    "description": "Get information about available flight routes. Can provide routes by region or direct routes for a specific city.",
    "parameters": {
        "type": "object",
        "properties": {
            "region_or_city": {
                "type": "string",
                "description": "Optional. Can be a region (like 'Europe') or a specific city. If not provided, returns all regions and destinations.",
            },
        },
        "required": [],
        "additionalProperties": False
    }
} 

In [15]:
# Updated tools list with all three functions

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": status_function},
    {"type": "function", "function": routes_function}
        ]

Getting OpenAI to use our Tool

In [16]:
def chat(message, history):
    # Prepare messages with system message and chat history
    messages = [{"role": "system", "content": system_message}]
    messages.extend(history)
    messages.append({"role": "user", "content": message})

    # First API call to potentially trigger tool calls
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=messages, 
        tools=tools,
        tool_choice="auto"  # Let the model decide when to use tools
    )

    # Check if tool calls are needed
    if response.choices[0].finish_reason == "tool_calls":
        # Get the message with tool calls
        message_with_tool_calls = response.choices[0].message
        
        # Prepare to collect tool responses
        messages.append(message_with_tool_calls)
        
        # Process each tool call
        for tool_call in message_with_tool_calls.tool_calls:
            arguments = json.loads(tool_call.function.arguments)
            
            # Determine which tool was called and get the result
            if tool_call.function.name == "get_ticket_price":
                city = arguments.get('destination_city')
                result = get_ticket_price(city)
                tool_response = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": f"The ticket price to {city} is {result}"
                }
            elif tool_call.function.name == "get_flight_status":
                city = arguments.get('destination_city')
                result = get_flight_status(city)
                tool_response = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": f"The flight status to {city} is {result}"
                }
            elif tool_call.function.name == "get_available_routes":
                region_or_city = arguments.get('region_or_city')
                result = get_available_routes(region_or_city)
                tool_response = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": f"Available routes: {result}"
                }
            
            # Add tool response to messages
            messages.append(tool_response)
        
        # Make final API call to get the response
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages
        )
    
    # Return the final response content
    return response.choices[0].message.content

In [17]:
# Launch Gradio interface
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


fTool get_available_routes called for {region_or_city}
Tool get_ticket_price called for Tunisia
fTool get_available_routes called for {region_or_city}
Tool get_ticket_price called for Tunis
tool get_flight_status called for Tunis
